In [3]:
import os
import numpy as np
from tensorflow.keras.layers import (Input, Conv2D, LeakyReLU, BatchNormalization, 
                                     Add, Flatten, Dense, UpSampling2D, Activation)
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths to datasets
TRAINING_DATASET_PATH = "F:\\Thapar\\Semester 6\\Projects\\Image Processing Project\\DIV2K_train_HR"
VALIDATION_DATASET_PATH = "F:\\Thapar\\Semester 6\\Projects\\Image Processing Project\\DIV2K_valid_HR"

In [4]:
# Load images
def load_images(path, size=(128, 128)):
    images = []
    for file_name in os.listdir(path):
        img_path = os.path.join(path, file_name)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=size)
            img = img_to_array(img) / 255.0
            images.append(img)
    return np.array(images)
# Load datasets
train_lr = load_images(TRAINING_DATASET_PATH, size=(64, 64))  # Low resolution
train_hr = load_images(TRAINING_DATASET_PATH, size=(128, 128))  # High resolution

In [5]:
# Define the generator
def build_generator():
    def residual_block(inputs):
        x = Conv2D(64, kernel_size=3, strides=1, padding='same')(inputs)
        x = BatchNormalization(momentum=0.8)(x)
        x = LeakyReLU(negative_slope=0.2)(x)
        x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
        x = BatchNormalization(momentum=0.8)(x)
        return Add()([inputs, x])

    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, kernel_size=9, strides=1, padding='same')(inputs)
    x = LeakyReLU(negative_slope=0.2)(x)

    # Residual blocks
    for _ in range(16):
        x = residual_block(x)

    # Upsampling layer (for 128x128 output)
    x = UpSampling2D(size=2)(x)  # From 64x64 to 128x128
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
    x = LeakyReLU(negative_slope=0.2)(x)

    # Final layer to produce 128x128x3 output
    outputs = Conv2D(3, kernel_size=9, strides=1, padding='same', activation='tanh')(x)
    return Model(inputs, outputs, name="Generator")

In [6]:
# Define the discriminator
def build_discriminator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(inputs)
    x = LeakyReLU(negative_slope=0.2)(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(negative_slope=0.2)(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(negative_slope=0.2)(x)

    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = LeakyReLU(negative_slope=0.2)(x)

    x = Flatten()(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs, name="Discriminator")

In [7]:
# Build and compile the models
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
# Define the updated generator using the new build_generator function
generator = build_generator()

# Freeze the discriminator while training the generator
discriminator.trainable = False

# Input low-resolution images
lr_inputs = Input(shape=(64, 64, 3))

# Generator outputs high-resolution images (128x128x3)
generated_hr = generator(lr_inputs)

# Discriminator evaluates the generated high-resolution images
validity = discriminator(generated_hr)

# Combine the generator and discriminator into the adversarial model
adversarial_model = Model(lr_inputs, [validity, generated_hr])

# Compile the adversarial model
adversarial_model.compile(
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
    loss=['binary_crossentropy', 'mse'],
    loss_weights=[1e-3, 1]
)

In [10]:
# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
print(discriminator.summary())  # This should print model architecture

Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_18 (LeakyReLU)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_19 (LeakyReLU)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_20 (LeakyReLU)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_21 (LeakyReLU)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,587,329 (6.06 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,587,329 (6.06 MB)

None


In [12]:
print(imgs_hr.shape)  # Shape of real images
print(fake_hr.shape)  # Shape of fake images

(32, 128, 128, 3)
(32, 128, 128, 3)


In [13]:
# Normalize images to range [-1, 1] if using tanh activation
train_lr = (train_lr.astype(np.float32) - 127.5) / 127.5
train_hr = (train_hr.astype(np.float32) - 127.5) / 127.5

In [15]:
import tensorflow.keras.backend as K

K.clear_session()  # Clears any previous model state


In [16]:
import tensorflow.keras.backend as K

K.clear_session()  # Correct way in TensorFlow 2.x

In [18]:
# Training parameters
batch_size = 32
epochs = 400

# Training loop
for epoch in range(epochs):
    # Sample random batch of images
    idx = np.random.randint(0, train_lr.shape[0], batch_size)
    imgs_lr, imgs_hr = train_lr[idx], train_hr[idx]

    # Generate high-resolution images
    fake_hr = generator.predict(imgs_lr)

    # Train the discriminator
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(imgs_hr, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_hr, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    g_loss = adversarial_model.train_on_batch(imgs_lr, [real_labels, imgs_hr])

    # Print progress
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"D Loss: {d_loss[0]:.4f}, D Accuracy: {100 * d_loss[1]:.2f}%, G Loss: {g_loss[0]:.4f}")

# Save the models
generator.save("generator_model.h5")
discriminator.save("discriminator_model.h5")
adversarial_model.save("adversarial_model.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 1/1000
D Loss: 0.6911, D Accuracy: 59.34%, G Loss: 0.3370
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 2/1000
D Loss: 0.6914, D Accuracy: 58.13%, G Loss: 0.2949
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 3/1000
D Loss: 0.6917, D Accuracy: 57.19%, G Loss: 0.2622
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 4/1000
D Loss: 0.6919, D Accuracy: 56.45%, G Loss: 0.2361
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 5/1000
D Loss: 0.6921, D Accuracy: 55.84%, G Loss: 0.2147
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 6/1000
D Loss: 0.6922, D Accuracy: 55.34%, G Loss: 0.1969
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 7/1000
D Loss: 0.6923, D Accuracy: 54.92%, G Loss: 0.1818
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 8/1000
D Loss: 0.6924, D Accuracy: 54.56%, G Loss: 0.1688
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 9/1000
D Loss: 0.6924, D Accuracy: 54.25%, G Loss: 0.1576
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 10/1000
D Loss: 0.6925, D Accuracy: 53.98%, G Loss: 0.1478

: 